## Run Once

In [0]:
# Used in many places
import psycopg2 as pg
import pandas as pd

# Used to enter database credentials without saving them to the notebook file
import getpass

# Used to easily read in bus location data
import pandas.io.sql as sqlio

# Only used in the schedule class definition
import numpy as np
from scipy import stats

# Used in the fcc_projection function to find distances
from math import sqrt, cos

In [16]:
# Enter database credentials.  Requires you to paste in the user and
# password so it isn't saved in the notebook file
print("Enter database username:")
user = getpass.getpass()
print("Enter database password:")
password = getpass.getpass()

creds = {
  'user': user,
  'password': password,
  'host': "lambdalabs24sfmta.cykkiwxbfvpg.us-east-1.rds.amazonaws.com",
  'dbname': "historicalTransitData"
}

# Set up connection to database
cnx = pg.connect(**creds)
cursor = cnx.cursor()

print('\nDatabase connection successful')

Enter database username:
··········
Enter database password:
··········

Database connection successful


In [0]:
# Schedule class definition
# Copied from previous work, has extra methods that are not all used in this notebook

class Schedule:
    def __init__(self, route_id, date, connection):
        """
        The Schedule class loads the schedule for a particular route and day,
        and makes several accessor methods available for it.

        Parameters:

        route_id (str or int)
            - The route id to load

        date (str or pandas.Timestamp)
            - Which date to load
            - Converted with pandas.to_datetime so many formats are acceptable
        """

        self.route_id = str(route_id)
        self.date = pd.to_datetime(date)

        # load the schedule for that date and route
        self.route_data = load_schedule(self.route_id, self.date, connection)

        # process data into a table
        self.inbound_table, self.outbound_table = extract_schedule_tables(self.route_data)

        # calculate the common interval values
        self.mean_interval, self.common_interval = get_common_intervals(
                                    [self.inbound_table, self.outbound_table])

    def list_stops(self):
        """
        returns the list of all stops used by this schedule
        """

        # get stops for both inbound and outbound routes
        inbound = list(self.inbound_table.columns)
        outbound = list(self.outbound_table.columns)

        # convert to set to ensure no duplicates,
        # then back to list for the correct output type
        return list(set(inbound + outbound))

    def get_specific_interval(self, stop, time, inbound=True):
        """
        Returns the expected interval, in minutes, for a given stop and
        time of day.

        Parameters:

        stop (str or int)
            - the stop tag/id of the bus stop to check

        time (str or pandas.Timestamp)
            - the time of day to check, uses pandas.to_datetime to convert
            - examples that work: "6:00", "3:30pm", "15:30"

        inbound (bool, optional)
            - whether to check the inbound or outbound schedule
            - ignored unless the given stop is in both inbound and outbound
        """

        # ensure correct parameter types
        stop = str(stop)
        time = pd.to_datetime(time)

        # check which route to use, and extract the column for the given stop
        if (stop in self.inbound_table.columns and
        stop in self.outbound_table.columns):
            # stop exists in both, use inbound parameter to decide
            if inbound:
                sched = self.inbound_table[stop]
            else:
                sched = self.outbound_table[stop]
        elif (stop in self.inbound_table.columns):
            # stop is in the inbound schedule, use that
            sched = self.inbound_table[stop]
        elif (stop in self.outbound_table.columns):
            # stop is in the outbound schedule, use that
            sched = self.outbound_table[stop]
        else:
            # stop doesn't exist in either, throw an error
            raise ValueError(f"Stop id '{stop}' doesn't exist in either inbound or outbound schedules")

        # 1: convert schedule to datetime for comparison statements
        # 2: drop any NaN values
        # 3: convert to list since pd.Series threw errors on i indexing
        sched = list(pd.to_datetime(sched).dropna())

        # reset the date portion of the time parameter to
        # ensure we are checking the schedule correctly
        time = time.replace(year=self.date.year, month=self.date.month,
                            day=self.date.day)

        # iterate through that list to find where the time parameter fits
        for i in range(1, len(sched)):
            # start at 1 and move forward,
            # is the time parameter before this schedule entry?
            if(time < sched[i]):
                # return the difference between this entry and the previous one
                return (sched[i] - sched[i-1]).seconds / 60

        # can only reach this point if the time parameter is after all entries
        # in the schedule, return the last available interval
        return (sched[len(sched)-1] - sched[len(sched)-2]).seconds / 60


def load_schedule(route, date, connection):
    """
    loads schedule data from the database and returns it

    Parameters:

        route (str)
            - The route id to load

        date (str or pd.Timestamp)
            - Which date to load
            - Converted with pandas.to_datetime so many formats are acceptable
    """

    # ensure correct parameter types
    route = str(route)
    date = pd.to_datetime(date)

    # DB connection
    cursor = connection.cursor()

    # build selection query
    query = """
        SELECT content
        FROM schedules
        WHERE rid = %s AND
            begin_date <= %s::TIMESTAMP AND
            (end_date IS NULL OR end_date >= %s::TIMESTAMP);
    """

    # execute query and save the route data to a local variable
    cursor.execute(query, (route, str(date), str(date)))
    data = cursor.fetchone()[0]['route']

    # pd.Timestamp.dayofweek returns 0 for monday and 6 for Sunday
    # the actual serviceClass strings are defined by Nextbus
    # these are the only 3 service classes we can currently observe,
    # if others are published later then this will need to change
    if(date.dayofweek <= 4):
        serviceClass = 'wkd'
    elif(date.dayofweek == 5):
        serviceClass = 'sat'
    else:
        serviceClass = 'sun'

    # the schedule format has two entries for each serviceClass,
    # one each for inbound and outbound.

    # return each entry in the data list with the correct serviceClass
    return [sched for sched in data if (sched['serviceClass'] == serviceClass)]


def extract_schedule_tables(route_data):
    """
    converts raw schedule data to two pandas dataframes

    columns are stops, and rows are individual trips

    returns inbound_df, outbound_df
    """

    # assuming 2 entries, but not assuming order
    if(route_data[0]['direction'] == 'Inbound'):
        inbound = 0
    else:
        inbound = 1

    # extract a list of stops to act as columns
    inbound_stops = [s['tag'] for s in route_data[inbound]['header']['stop']]

    # initialize dataframe
    inbound_df = pd.DataFrame(columns=inbound_stops)

    # extract each row from the data
    if type(route_data[inbound]['tr']) == list:
        # if there are multiple trips in a day, structure will be a list
        i = 0
        for trip in route_data[inbound]['tr']:
            for stop in trip['stop']:
                # '--' indicates the bus is not going to that stop on this trip
                if stop['content'] != '--':
                    inbound_df.at[i, stop['tag']] = stop['content']
            # increment for the next row
            i += 1
    else:
        # if there is only 1 trip in a day, the object is a dict and
        # must be handled slightly differently
        for stop in route_data[inbound]['tr']['stop']:
            if stop['content'] != '--':
                    inbound_df.at[0, stop['tag']] = stop['content']

    # flip between 0 and 1
    outbound = int(not inbound)

    # repeat steps for the outbound schedule
    outbound_stops = [s['tag'] for s in route_data[outbound]['header']['stop']]
    outbound_df = pd.DataFrame(columns=outbound_stops)

    if type(route_data[outbound]['tr']) == list:
        i = 0
        for trip in route_data[outbound]['tr']:
            for stop in trip['stop']:
                if stop['content'] != '--':
                    outbound_df.at[i, stop['tag']] = stop['content']
            i += 1
    else:
        for stop in route_data[outbound]['tr']['stop']:
            if stop['content'] != '--':
                    outbound_df.at[0, stop['tag']] = stop['content']

    # return both dataframes
    return inbound_df, outbound_df


def get_common_intervals(df_list):
    """
    takes route schedule tables and returns both the average interval (mean)
    and the most common interval (mode), measured in number of minutes

    takes a list of dataframes and combines them before calculating statistics

    intended to combine inbound and outbound schedules for a single route
    """

    # ensure we have at least one dataframe
    if len(df_list) == 0:
        raise ValueError("Function requires at least one dataframe")

    # append all dataframes in the array together
    df = df_list[0].copy()
    for i in range(1, len(df_list)):
        df.append(df_list[i].copy())

    # convert all values to datetime so we can get an interval easily
    for col in df.columns:
        df[col] = pd.to_datetime(df[col])

    # initialize a table to hold each individual interval
    intervals = pd.DataFrame(columns=df.columns)
    intervals['temp'] = range(len(df))

    # take each column and find the intervals in it
    for col in df.columns:
        prev_time = np.nan
        for i in range(len(df)):
            # find the first non-null value and save it to prev_time
            if pd.isnull(prev_time):
                prev_time = df.at[i, col]
            # if the current time is not null, save the interval
            elif ~pd.isnull(df.at[i, col]):
                intervals.at[i, col] = (df.at[i, col] - prev_time).seconds / 60
                prev_time = df.at[i, col]

    # this runs without adding a temp column, but the above loop runs 3x as
    # fast if the rows already exist
    intervals = intervals.drop('temp', axis=1)

    # calculate the mean of the entire table
    mean = intervals.mean().mean()

    # calculate the mode of the entire table, the [0][0] at the end is
    # because scipy.stats returns an entire ModeResult class
    mode = stats.mode(intervals.values.flatten())[0][0]

    return mean, mode

In [0]:
# Route class definition
# Copied from previous work, has extra methods that are not all used in this notebook

class Route:
    def __init__(self, route_id, date, connection):
        """
        The Route class loads the route configuration data for a particular
        route, and makes several accessor methods available for it.

        Parameters:

        route_id (str or int)
            - The route id to load

        date (str or pandas.Timestamp)
            - Which date to load
            - Converted with pandas.to_datetime so many formats are acceptable
        """

        self.route_id = str(route_id)
        self.date = pd.to_datetime(date)

        # load the route data
        self.route_data, self.route_type, self.route_name = load_route(self.route_id, self.date, connection)

        # extract stops info and rearrange columns to be more human readable
        # note: the stop tag is what was used in the schedule data, not stopId
        self.stops_table = pd.DataFrame(self.route_data['stop'])
        self.stops_table = self.stops_table[['stopId', 'tag', 'title', 'lat', 'lon']]

        # extract route path, list of (lat, lon) pairs
        self.path_coords = extract_path(self.route_data)

        # extract stops table
        self.stops_table, self.inbound, self.outbound = extract_stops(self.route_data)


def load_route(route, date, connection):
    """
    loads raw route data from the database

    Parameters:

        route (str or int)
            - The route id to load

        date (str or pd.Timestamp)
            - Which date to load
            - Converted with pandas.to_datetime so many formats are acceptable
    
    Returns route_data (dict), route_type (str), route_name (str)
    """

    # ensure correct parameter types
    route = str(route)
    date = pd.to_datetime(date)

    # DB connection
    cursor = connection.cursor()

    # build selection query
    query = """
        SELECT route_name, route_type, content
        FROM routes
        WHERE rid = %s AND
            begin_date <= %s::TIMESTAMP AND
            (end_date IS NULL OR end_date > %s::TIMESTAMP);
    """

    # execute query and return the route data
    cursor.execute(query, (route, str(date), str(date)))
    result = cursor.fetchone()
    return result[2]['route'], result[1], result[0]


def extract_path(route_data):
    """
    Extracts the list of path coordinates for a route.

    The raw data stores this as an unordered list of sub-routes, so this
    function deciphers the order they should go in and returns a single list.
    """

    # KNOWN BUG
    # this approach assumed all routes were either a line or a loop.
    # routes that have multiple sub-paths meeting at a point break this,
    # route 24 is a current example.
    # I'm committing this now to get the rest of the code out there

    # extract the list of subpaths as just (lat,lon) coordinates
    # also converts from string to float (raw data has strings)
    path = []
    for sub_path in route_data['path']:
        path.append([(float(p['lat']), float(p['lon'])) 
                     for p in sub_path['point']])

    # start with the first element, remove it from path
    final = path[0]
    path.pop(0)

    # loop until the first and last coordinates in final match
    counter = len(path)
    done = True
    while final[0] != final[-1]:
        # loop through the sub-paths that we haven't yet moved to final
        for i in range(len(path)):
            # check if the last coordinate in final matches the first 
            # coordinate of another sub-path
            if final[-1] == path[i][0]:
                # match found, move it to final
                # leave out the first coordinate to avoid duplicates
                final = final + path[i][1:]
                path.pop(i)
                break  # break the for loop
                
        # protection against infinite loops, if the path never closes
        counter -= 1
        if counter < 0:
            done = False
            break

    if not done:
        # route did not connect in a loop, perform same steps backwards 
        # to get the rest of the line
        for _ in range(len(path)):
            # loop through the sub-paths that we haven't yet moved to final
            for i in range(len(path)):
                # check if the first coordinate in final matches the last 
                # coordinate of another sub-path
                if final[0] == path[i][-1]:
                    # match found, move it to final
                    # leave out the last coordinate to avoid duplicates
                    final = path[i][:-1] + final
                    path.pop(i)
                    break  # break the for loop

    # some routes may have un-used sub-paths
    # Route 1 for example has two sub-paths that are almost identical, with the 
    # same start and end points
    # if len(path) > 0:
    #     print(f"WARNING: {len(path)} unused sub-paths")

    # return the final result
    return final


def extract_stops(route_data):
  """
  Extracts a dataframe of stops info

  Returns the main stops dataframe, and a list of inbound and outbound stops 
  in the order they are intended to be on the route
  """

  stops = pd.DataFrame(route_data['stop'])
  directions = pd.DataFrame(route_data['direction'])

  # Change stop arrays to just the list of numbers
  for i in range(len(directions)):
    directions.at[i, 'stop'] = [s['tag'] for s in directions.at[i, 'stop']]

  # Find which stops are inbound or outbound
  inbound = []
  for stop_list in directions[directions['name'] == "Inbound"]['stop']:
    for stop in stop_list:
      if stop not in inbound:
        inbound.append(stop)

  outbound = []
  for stop_list in directions[directions['name'] == "Outbound"]['stop']:
    for stop in stop_list:
      if stop not in inbound:
        outbound.append(stop)

  # Label each stop as inbound or outbound
  stops['direction'] = ['none'] * len(stops)
  for i in range(len(stops)):
    if stops.at[i, 'tag'] in inbound:
      stops.at[i, 'direction'] = 'inbound'
    elif stops.at[i, 'tag'] in outbound:
      stops.at[i, 'direction'] = 'outbound'

  # Convert from string to float
  stops['lat'] = stops['lat'].astype(float)
  stops['lon'] = stops['lon'].astype(float)

  return stops, inbound, outbound

In [0]:
def get_location_data(rid, begin, end, connection):
  # Build query to select location data
  query = f"""
    SELECT *
    FROM locations
    WHERE rid = '{rid}' AND
      timestamp > '{begin}'::TIMESTAMP AND
      timestamp < '{end}'::TIMESTAMP
    ORDER BY id;
  """

  # read the query directly into pandas
  locations = sqlio.read_sql_query(query, connection)

  # Convert those UTC timestamps to local PST by subtracting 7 hours
  locations['timestamp'] = locations['timestamp'] - pd.Timedelta(hours=7)

  # return the result
  return locations


In [0]:
# Written by Austie
def fcc_projection(loc1, loc2):
    """
    function to apply FCC recommended formulae
    for calculating distances on earth projected to a plane
    
    significantly faster computationally, negligible loss in accuracy
    
    Args: 
    loc1 - a tuple of lat/lon
    loc2 - a tuple of lat/lon
    """
    lat1, lat2 = loc1[0], loc2[0]
    lon1, lon2 = loc1[1], loc2[1]
    
    mean_lat = (lat1+lat2)/2
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    
    k1 = 111.13209 - 0.56605*cos(2*mean_lat) + .0012*cos(4*mean_lat)
    k2 = 111.41513*cos(mean_lat) - 0.09455*cos(3*mean_lat) + 0.00012*cos(5*mean_lat)
    
    distance = sqrt((k1*delta_lat)**2 + (k2*delta_lon)**2)
    
    return distance


In [0]:
def clean_locations(locations, stops):
  """
  takes a dataframe of bus locations and a dataframe of 

  returns the locations dataframe with nearest stop added
  """
  
  # remove old location reports that would be duplicates
  df = locations[locations['age'] < 60].copy()

  # remove rows with no direction value
  df = df[~pd.isna(df['direction'])]

  # shift timestamps according to the age column
  df['timestamp'] = df.apply(shift_timestamp, axis=1)

  # Make lists of all inbound or outbound stops
  inbound_stops = stops[stops['direction'] == 'inbound'].reset_index(drop=True)
  outbound_stops = stops[stops['direction'] == 'outbound'].reset_index(drop=True)

  # initialize new columns for efficiency
  df['closestStop'] = [0] * len(df)
  df['distance'] = [0.0] * len(df)

  for i in df.index:
    if '_I_' in df.at[i, 'direction']:
      candidates = inbound_stops
    elif '_O_' in df.at[i, 'direction']:
      candidates = outbound_stops
    else:
      # Skip row if bus is not found to be either inbound or outbound
      continue
    
    bus_coord = (df.at[i, 'latitude'], df.at[i, 'longitude'])

    # Find closest stop within candidates
    # Assume the first stop
    closest = candidates.iloc[0]
    distance = fcc_projection(bus_coord, (closest['lat'], closest['lon']))

    # Check each stop after that
    for _, row in candidates[1:].iterrows():
      # find distance to this stop
      dist = fcc_projection(bus_coord, (row['lat'], row['lon']))
      if dist < distance:
        # closer stop found, save it
        closest = row
        distance = dist
    
    # Save the tag of the closest stop and the distance to it
    df.at[i, 'closestStop'] = closest['tag']
    df.at[i, 'distance'] = distance

  return df


def shift_timestamp(row):
  """ subtracts row['age'] from row['timestamp'] """
  return row['timestamp'] - pd.Timedelta(seconds=row['age'])

In [0]:
def get_stop_times(locations, route):
  """
  returns a dict, keys are stop tags and values are lists of timestamps 
  that describe every time a bus was seen at that stop
  """
  start = time()
  # Initialize the data structure I will store results in
  stop_times = {}
  vids = {}
  for stop in route.inbound + route.outbound:
    stop_times[str(stop)] = []

  for vid in locations['vid'].unique():
    # Process the route one vehicle at a time
    df = locations[locations['vid'] == vid]

    # process 1st row on its own
    prev_row = df.loc[df.index[0]]
    stop_times[str(prev_row['closestStop'])].append(prev_row['timestamp'])

    # loop through the rest of the rows, comparing each to the previous one
    for i, row in df[1:].iterrows():
      if row['direction'] != prev_row['direction']:
        # changed directions, don't compare to previous row
        stop_times[str(row['closestStop'])].append(row['timestamp'])
      else:
        # same direction, compare to previous row
        if '_I_' in row['direction']:  # get correct stop list
          stoplist = route.inbound
        else:
          stoplist = route.outbound

        current = stoplist.index(str(row['closestStop']))
        previous = stoplist.index(str(prev_row['closestStop']))
        gap = current - previous
        if gap > 1:  # need to interpolate
          diff = (row['timestamp'] - prev_row['timestamp'])/gap
          counter = 1
          for stop in stoplist[previous+1:current]:
            # save interpolated time
            stop_times[str(stop)].append(prev_row['timestamp'] + (counter * diff))

            # increase counter for the next stop
            # example: with 2 interpolated stops, gap would be 3
            # 1st diff is 1/3, next is 2/3
            counter += 1
        
        if row['closestStop'] != prev_row['closestStop']:
          # only save time if the stop has changed, 
          # otherwise the bus hasn't moved since last time
          stop_times[str(row['closestStop'])].append(row['timestamp'])
      
      # advance for next row
      prev_row = row

  # Sort each list before returning
  for stop in stop_times.keys():
    stop_times[stop].sort()

  return stop_times


In [0]:
def get_bunches_gaps(stop_times, schedule, bunch_threshold=.2, gap_threshold=1.5):
  """
  returns a dataframe of all bunches and gaps found

  default thresholds define a bunch as 20% and a gap as 150% of scheduled headway
  """

  # Initialize dataframe for the bunces and gaps
  problems = pd.DataFrame(columns=['type', 'time', 'duration', 'stop'])
  counter = 0

  # Set the bunch/gap thresholds (in seconds)
  bunch_threshold = (schedule.common_interval * 60) * bunch_threshold
  gap_threshold = (schedule.common_interval * 60) * gap_threshold

  for stop in stop_times.keys():
    # ensure we have any times at all for this stop
    if len(stop_times[stop]) == 0:
      #print(f"Stop {stop} had no recorded times")
      continue  # go to next stop in the loop

    # save initial time
    prev_time = stop_times[stop][0]

    # loop through all others, comparing to the previous one
    for time in stop_times[stop][1:]:
      diff = (time - prev_time).seconds
      if diff <= bunch_threshold:
        # bunch found, save it
        problems.at[counter] = ['bunch', prev_time, diff, stop]
        counter += 1
      elif diff >= gap_threshold:
        problems.at[counter] = ['gap', prev_time, diff, stop]
        counter += 1
      
      prev_time = time
  
  return problems


In [0]:
# this uses sequential search, could speed up with binary search if needed,
# but it currently uses hardly any time in comparison to other steps
def helper_count(expected_times, observed_times):
  """ Returns the number of on-time stops found """

  # set up early/late thresholds (in seconds)
  early_threshold = pd.Timedelta(seconds=1*60)  # 1 minute early
  late_threshold = pd.Timedelta(seconds=4*60)   # 4 minutes late

  count = 0
  for stop in expected_times.columns:
    for expected in expected_times[stop]:
      if pd.isna(expected):
        continue  # skip NaN values in the expected schedule

      # for each expected time...
      # find first observed time after the early threshold
      found_time = None
      early = expected - early_threshold

      # BUG: some schedule data may have stop tags that are not in the inbound
      # or outbound definitions for a route.  That would throw a key error here.
      # Example: stop 14148 on route 24
      # current solution is to ignore those stops with the try/except statement
      try:
        for observed in observed_times[stop]:
          if observed >= early:
            found_time = observed
            break
      except:
        continue

      # if found time is still None, then all observed times were too early
      # if found_time is before the late threshold then we were on time
      if (not pd.isna(found_time)) and found_time <= (expected + late_threshold):
        # found_time is within the on-time window
        count += 1

  return count

def calculate_ontime(stop_times, schedule):
  """ Returns the on-time percentage and total scheduled stops for this route """

  # Save schedules with timestamp data types, set date to match
  inbound_times = schedule.inbound_table
  for col in inbound_times.columns:
    inbound_times[col] = pd.to_datetime(inbound_times[col]).apply(
        lambda dt: dt.replace(year=schedule.date.year, 
                              month=schedule.date.month, 
                              day=schedule.date.day))

  outbound_times = schedule.outbound_table
  for col in outbound_times.columns:
    outbound_times[col] = pd.to_datetime(outbound_times[col]).apply(
        lambda dt: dt.replace(year=schedule.date.year, 
                              month=schedule.date.month, 
                              day=schedule.date.day))
  
  # count times for both inbound and outbound schedules
  on_time_count = (helper_count(inbound_times, stop_times) +
                   helper_count(outbound_times, stop_times))
  
  # get total expected count
  total_expected = inbound_times.count().sum() + outbound_times.count().sum()

  # return on-time percentage
  return (on_time_count / total_expected), total_expected


In [0]:
def bunch_gap_graph(problems, interval=10):
  """
  returns data for a graph of the bunches and gaps throughout the day

  problems - the dataframe of bunches and gaps

  interval - the number of minutes to bin data into

  returns
  {
    "times": [time values (x)],
    "bunches": [bunch counts (y1)],
    "gaps": [gap counts (y2)]
  }
  """

  # set the time interval
  interval = pd.Timedelta(minutes=interval)

  # rest of code doesn't work if there are no bunches or gaps
  # return the empty graph manually
  if len(problems) == 0:
    # generate list of times according to the interval
    start = pd.Timestamp('today').replace(hour=0, minute=0, second=0)
    t = start
    times = []
    while t.day == start.day:
      times.append(str(t.time())[:5])
      t += interval

    return {
      "times": times,
      "bunches": [0] * len(times),
      "gaps": [0] * len(times)
    }

  # generate the DatetimeIndex needed
  index = pd.DatetimeIndex(problems['time'])
  df = problems.copy()
  df.index = index

  # lists for graph data
  bunches = []
  gaps = []
  times = []
  
  # set selection times
  start_date = problems.at[0, 'time'].replace(hour=0, minute=0, second=0)
  select_start = start_date
  select_end = select_start + interval

  while select_start.day == start_date.day:
    # get the count of each type of problem in this time interval
    count = df.between_time(select_start.time(), select_end.time())['type'].value_counts()

    # append the counts to the data list
    if 'bunch' in count.index:
      bunches.append(int(count['bunch']))
    else:
      bunches.append(0)
    
    if 'gap' in count.index:
      gaps.append(int(count['gap']))
    else:
      gaps.append(0)

    # save the start time for the x axis
    times.append(str(select_start.time())[:5])
    
    # increment the selection window
    select_start += interval
    select_end += interval
  
  return {
    "times": times,
    "bunches": bunches,
    "gaps": gaps
  }

## Necessary for geojson

In [0]:
def create_simple_geojson(bunches, rid):

    geojson = {'type': 'FeatureCollection',
               'bunches': create_geojson_features(bunches, rid)}

    return geojson

In [0]:
def create_geojson_features(df, rid):
    """
    function to generate list of geojson features
    for plotting vehicle locations on timestamped map

    Expects a dataframe containing lat/lon, vid, timestamp
    returns list of basic geojson formatted features:

    {
      type: Feature
      geometry: {
        type: Point,
        coordinates:[lat, lon]
      },
      properties: {
        route_id: rid
        time: timestamp
      }
    }
    """
    # initializing empty features list
    features = []

    # iterating through df to pull coords, vid, timestamp
    # and format for json
    for index, row in df.iterrows():
      feature = {
          'type': 'Feature',
          'geometry': {
              'type':'Point', 
              'coordinates':[row.lon, row.lat]
          },
          'properties': {
              'time': row.time.__str__(),
              'stop': {'stopId': row.stopId.__str__(),
                       'stopTitle': row.title.__str__()},
              'direction': row.direction.__str__()
          }
      }
      features.append(feature) # adding point to features list
    return features

## Generating report JSON

Now updated to include geojson for mapping.

Tested geojson generation with: 
- single report generation 
- all routes generation
- aggregate generation

Tested mapping bunches with generated geojson W/ Folium. 

Everything should plug-and-play.

In [0]:
def generate_report(rid, date):
  """
  Generates a daily report for the given rid and date

  rid : (str)
    the route id to generate a report for
  
  date : (str or pd.Datetime)
    the date to generate a report for

  returns a dict of the report info
  """

  # get begin and end timestamps for the date
  begin = pd.to_datetime(date).replace(hour=7)
  end = begin + pd.Timedelta(days=1)
  # Load schedule and route data
  schedule = Schedule(rid, begin, cnx)
  
  route = Route(rid, begin, cnx)
  
  # Load bus location data
  locations = get_location_data(rid, begin, end, cnx)
  
  # Apply cleaning function (this usually takes 1-2 minutes)
  locations = clean_locations(locations, route.stops_table)
  
  # Calculate all times a bus was at each stop
  stop_times = get_stop_times(locations, route)

  # Find all bunches and gaps
  problems = get_bunches_gaps(stop_times, schedule)

  # Calculate on-time percentage
  on_time, total_scheduled = calculate_ontime(stop_times, schedule)

  # Build result dict
  count_times = 0
  for key in stop_times.keys():
    count_times += len(stop_times[key])

  # Number of recorded intervals ( sum(len(each list of time)) - number or lists of times)
  intervals = count_times-len(stop_times)

  bunches = len(problems[problems['type'] == 'bunch'])
  gaps = len(problems[problems['type'] == 'gap'])

  coverage = (total_scheduled * on_time + bunches) / total_scheduled
  
  # Isolating bunches, merging with stops to assign locations to bunches
  stops = route.stops_table.copy()

  bunch_df = problems[problems.type.eq('bunch')]
  bunch_df = bunch_df.merge(stops, left_on='stop', right_on='tag', how='left')

  # Creating GeoJSON of bunch times / locations
  geojson = create_simple_geojson(bunch_df, rid)
  
  # int/float conversions are because the json library doesn't work with numpy types
  result = {
      'route': rid,
      'route_name': route.route_name,
      'route_type': route.route_type,
      'date': str(pd.to_datetime(date)),
      'num_bunches': bunches,
      'num_gaps': gaps,
      'total_intervals': intervals,
      'on_time_percentage': float(round(on_time * 100, 2)),
      'scheduled_stops': int(total_scheduled),
      'coverage': float(round(coverage * 100, 2)),
      # line_chart contains all data needed to generate the line chart
      'line_chart': bunch_gap_graph(problems, interval=10),
      # route_table is an array of all rows that should show up in the table
      # it will be filled in after all reports are generated
      'route_table': [
          {
            'route_name': route.route_name,
            'bunches': bunches,
            'gaps': gaps,
            'on-time': float(round(on_time * 100, 2)),
            'coverage': float(round(coverage * 100, 2))
          }
      ],
      'geojson': geojson
  }

  return result

In [565]:
%%time

report_1 = generate_report(rid='1', date='2020/6/1')

CPU times: user 1min 12s, sys: 38.7 ms, total: 1min 12s
Wall time: 1min 16s


In [464]:
report_1['geojson']['bunches'][:5]

[{'geometry': {'coordinates': [-122.49311, 37.7797399], 'type': 'Point'},
  'properties': {'direction': 'inbound',
   'stop': {'stopId': '14277', 'stopTitle': 'Geary Blvd & 33rd Ave'},
   'time': '2020-06-01 13:44:03'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-122.49311, 37.7797399], 'type': 'Point'},
  'properties': {'direction': 'inbound',
   'stop': {'stopId': '14277', 'stopTitle': 'Geary Blvd & 33rd Ave'},
   'time': '2020-06-01 14:34:38'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-122.49311, 37.7797399], 'type': 'Point'},
  'properties': {'direction': 'inbound',
   'stop': {'stopId': '14277', 'stopTitle': 'Geary Blvd & 33rd Ave'},
   'time': '2020-06-01 21:49:39'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-122.49335, 37.78154], 'type': 'Point'},
  'properties': {'direction': 'inbound',
   'stop': {'stopId': '13555', 'stopTitle': '33rd Ave & Clement St'},
   'time': '2020-06-01 13:58:05'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-12

In [0]:
report_714 = generate_report(rid='714', date='2020/6/1')

In [471]:
report_714['geojson']['bunches']

[]

# Generating report for all routes

In [0]:
def get_active_routes(date):
  """
  returns a list of all active route id's for the given date
  """

  query = """
    SELECT DISTINCT rid
    FROM routes
    WHERE begin_date <= %s ::TIMESTAMP AND
          (end_date IS NULL OR end_date > %s ::TIMESTAMP);
  """

  cursor.execute(query, (date, date))
  return [result[0] for result in cursor.fetchall()]

In [558]:
%%time
# since this is not optimized yet, this takes about 20 minutes

# choose a day
date = '2020-6-1'

# get all active routes 
route_ids = get_active_routes(date)

# get the report for all routes
all_reports = []
for rid in route_ids:
  try:
    all_reports.append(generate_report(rid, date))
    print("Generated report for route", rid)
  except: # in case any particular route throws an error
    print(f"Route {rid} failed")

Generated report for route 1
Generated report for route NBUS
CPU times: user 2min 40s, sys: 97.7 ms, total: 2min 41s
Wall time: 2min 48s


In [559]:
len(all_reports)

2

In [0]:
# generate aggregate reports

# read existing reports into a dataframe to work with them easily
df = pd.DataFrame(all_reports)

# for each aggregate type
types = list(df['route_type'].unique()) + ['All']
for t in types:
  # filter df to the routes we are adding up
  if t == 'All':
    filtered = df
  else:
    filtered = df[df['route_type'] == t]

  # on-time percentage: sum([all on-time stops]) / sum([all scheduled stops])
  count_on_time = (filtered['on_time_percentage'] * filtered['scheduled_stops']).sum()
  on_time_perc = count_on_time / filtered['scheduled_stops'].sum()

  # coverage: (sum([all on-time stops]) + sum([all bunches])) / sum([all scheduled stops])
  coverage = (count_on_time + filtered['num_bunches'].sum()) / filtered['scheduled_stops'].sum()

  # aggregate the graph object
  # x-axis is same for all
  first = filtered.index[0]
  times = filtered.at[first, 'line_chart']['times']

  # sum up all y-axis values
  bunches = pd.Series(filtered.at[first, 'line_chart']['bunches'])
  gaps = pd.Series(filtered.at[first, 'line_chart']['gaps'])

  for chart in filtered[1:]['line_chart']:
    bunches += pd.Series(chart['bunches'])
    gaps += pd.Series(chart['gaps'])

  # save a new report object
  new_report = {
      'route': t,
      'route_name': t,
      'route_type': t,
      'date': all_reports[0]['date'],
      'num_bunches': int(filtered['num_bunches'].sum()),
      'num_gaps': int(filtered['num_gaps'].sum()),
      'total_intervals': int(filtered['total_intervals'].sum()),
      'on_time_percentage': float(round(on_time_perc, 2)),
      'scheduled_stops': int(filtered['scheduled_stops'].sum()),
      'coverage': float(round(coverage, 2)),
      'line_chart': {
          'times': times,
          'bunches': list(bunches),
          'gaps': list(bunches)
      },
      'route_table': [
          {
            'route_name': t,
            'bunches': int(filtered['num_bunches'].sum()),
            'gaps': int(filtered['num_gaps'].sum()),
            'on-time': float(round(on_time_perc, 2)),
            'coverage': float(round(coverage, 2))
          }
      ]
  }

  # TODO: add route_table rows to the aggregate report

  all_reports.append(new_report)

In [564]:
all_reports[0].keys()

dict_keys(['route', 'route_name', 'route_type', 'date', 'num_bunches', 'num_gaps', 'total_intervals', 'on_time_percentage', 'scheduled_stops', 'coverage', 'line_chart', 'route_table', 'geojson'])